In [1]:
import os
import json
import re
import pandas as pd
import numpy as np

from google.cloud import storage
from google.cloud import documentai_v1beta2 as documentai

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\ksaluj1\Documents\work\dev\gv\iacoe_sa.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/hsingh151/My_Work/Uber_IACOE_OCR_ML_Pipeline_v1.1.0/iacoe_sa.json"
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000

def parse_ner_json_to_string(parsed_ner):
    ner_string = ""
    flag = True
    for key, value in parsed_ner.items():
        if flag:
            ner_string = ner_string + value + "|" + key
            flag = False
        else:
            ner_string = ner_string + ";" + value + "|" + key
    return ner_string

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    file_name = source_file_name.split('/')[-1]
    destination_blob_name = destination_blob_name + "/" + file_name
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)
    gs_bucket_url = "gs://iacoe-cloud-bucket"
    gcs_uri = gs_bucket_url + '/' + destination_blob_name
    return gcs_uri

def extract_data_doc_ai(input_uri, project_id='concrete-tuner-241417'):
         # input_uri='gs://iacoe-cloud-bucket/k'):
    """Process a single document with the Document AI API, including
    text extraction and entity extraction."""

    client = documentai.DocumentUnderstandingServiceClient()
    gcs_source = documentai.types.GcsSource(uri=input_uri)

    input_config = documentai.types.InputConfig(
        gcs_source=gcs_source, mime_type='application/pdf')

    form_extraction_params = documentai.types.FormExtractionParams(
        enabled=True)
    table_extraction_params = documentai.types.TableExtractionParams(
        enabled=True)
    entity_extraction_params = documentai.types.EntityExtractionParams(
        enabled=True)

    parent = 'projects/{}/locations/us'.format(project_id)
    request = documentai.types.ProcessDocumentRequest(
        parent=parent,
        input_config=input_config,
        form_extraction_params=form_extraction_params,
        table_extraction_params=table_extraction_params,
        entity_extraction_params=entity_extraction_params)

    document = client.process_document(request=request)
    data = {}
    data["raw_text"] = (document.text).encode('utf-8')

    def _get_text(el):
        """Convert text offset indexes into text snippets.
        """
        response = ''
        for segment in el.text_anchor.text_segments:
            start_index = segment.start_index
            end_index = segment.end_index
            response += document.text[start_index:end_index]
        return response
    ent_dict = {}
    for entity in document.entities:
        ent_dict[_get_text(entity)] = (entity.mention_text)

    data["entities"] = ent_dict
    form_ent = {}
    for page in document.pages:
        for form_field in page.form_fields:
            key = (_get_text(form_field.field_name), form_field.field_name.confidence)
            value = (_get_text(form_field.field_value), form_field.field_value.confidence)
            form_ent[key] = value
    data["form_data"] = form_ent
    
    out_tables = []
    for page in document.pages:
        for table_num, table in enumerate(page.tables):
            out_table = []
            for row_num, row in enumerate(table.header_rows):
                out_header_row = [_get_text(cell.layout) for cell in row.cells]
                out_table.append(out_header_row)
            for row_num, row in enumerate(table.body_rows):
                out_body_row = [_get_text(cell.layout) for cell in row.cells]
                out_table.append(out_body_row)
            out_tables.append(out_table)

    data["tables"] = out_tables
    
    labels = {}
    for label in document.labels:
        labels[label.name] = label.confidence
    data["labels"] = labels

    return data

In [2]:
pwd

'C:\\Users\\hsingh151\\My_Work\\BOL\\03_Niagara_bol_processing\\04_jupyter_notebook'

In [3]:
import time
import os

In [4]:
data_files = {}

In [5]:
GCS_BUCKET = "iacoe-cloud-bucket"
DESTINATION_BLOB_NAME = "iacoe_ocr_ml"
file_loc = 'C:/Users/hsingh151/My_Work/BOL/03_Niagara_bol_processing/05_total_test_samples/'

arr_txt =  [x for x in os.listdir(file_loc) if x.endswith(".pdf") or x.endswith(".PDF")]
t1 = time.time()

for i in arr_txt:
    file_path = file_loc + i
    t3 = time.time()
    gcs_uri = upload_blob(GCS_BUCKET, file_path, DESTINATION_BLOB_NAME)
    data_files[i] = extract_data_doc_ai(gcs_uri)
    print("File {} : time {}".format(i, time.time() - t3))
    
t2 = time.time()

File 001_Niagara Template 1_Annotated.pdf : time 14.748740911483765
File 002_Pepsi-Niagara Format_Annotated.pdf : time 10.040374517440796
File 003_Niagara-Cocacola sample_Annotated.pdf : time 8.459882497787476


In [6]:
def parse_ner_json_to_string(parsed_ner):
    ner_string = ""
    flag = True
    print(parsed_ner)
    for key, value in parsed_ner.items():
        if flag:
            ner_string = ner_string + value + "|" + key
            flag = False
        else:
            ner_string = ner_string + ";" + value + "|" + key
    return ner_string

In [15]:
data_files[[*data_files][2]]['raw_text']

b'nlagara\nSTRAIGHT BILL OF LADING - SHORT FORM - Not negotiable\nNiagara Bottling, LLC\nBILL OF LADING\n1 OF 1\nSHIP FROM\n30272003\nBill of Lading Number:\nMaster Bill of Lading Number:\nName:\nNiagara Kenosha\n49842892\nAddress:\n11031 88th Avenue\nCustomer Po:\n4503488435\n0403485252\nCity/State/Zip:\nPleasant Prairie, WI 53158\nPRO/Reference #:\nDelivery #:\nShipmentNumber # :\n30272003\nSID#:\n1575\nFOB\n49842892\nSHIP TO\nCARRIER\nName:\nCOCA-COLA NAOU\nCCNA CPU UBER FREIGHT\nCarrier Name\nAdresses\n1455 MARKET ST 4TH FL\nAddress:\n105 INDUSTRIAL DR\nCity/State/Zip:\nSAN FRANCISCO\nCity/State/Zip:\nSAINT CHARLES, IL 60174\nSCAC\nCC33\nCID#:\n1517711\nFOB \xe2\x98\x90\nTrailer Number\n53606\nSeal #\n12861364\nFreight Charge Terms: (freight charges are prepaid unless marked\notherwise)\nPrepaid\nCollect\n3th Party\nCustomer Pickup\nPOD INSTRUCTIONS: Carrier FAX TO:(909) 494-4456 Or Email To Orders@niagarawater.com\nCustomer Order Information\nCustomer ID Item Description\nQuantity

In [14]:
data_files[[*data_files][2]]['entities']

{'Bill of Lading Number:\n30272003': '30272003',
 'Master Bill of Lading Number:\n49842892': '49842892',
 'Name:\nNiagara Kenosha': 'Niagara Kenosha',
 'Address:\n11031 88th Avenue': '11031 88th Avenue',
 'Customer Po:\n4503488435': '4503488435',
 'PRO/Reference #:\n0403485252': '0403485252',
 'City/State/Zip:\nPleasant Prairie, WI 53158': 'Pleasant Prairie, WI 53158',
 'Delivery #:\n30272003': '30272003',
 'ShipmentNumber # :\n49842892': '49842892',
 'SID#:\n1575': '1575',
 'Name:\nCOCA-COLA NAOU': 'COCA-COLA NAOU',
 'Carrier Name\nCCNA CPU UBER FREIGHT': 'CCNA CPU UBER FREIGHT',
 'Adresses\n1455 MARKET ST 4TH FL': '1455 MARKET ST 4TH FL',
 'Address:\n105 INDUSTRIAL DR': '105 INDUSTRIAL DR',
 'City/State/Zip:\nSAN FRANCISCO': 'SAN FRANCISCO',
 'City/State/Zip:\nSAINT CHARLES, IL 60174': 'SAINT CHARLES, IL 60174',
 'SCAC\nCC33': 'CC33',
 'CID#:\n1517711': '1517711',
 'Trailer Number\n53606': '53606',
 'Seal #\n12861364': '12861364',
 'Customer ID 707203': '707203',
 'Item Description\n

In [ ]:
def niagara_format1_doc_ai_ner_parser(raw_text, entities_json):
    parsed_data = {}
    
    #Look up keys
    Bill_of_Lading_Number_Key = "Bill of Lading Number:"
    Master_Bill_of_Lading_Number_Key = "Master Bill of Lading Number:"
    Customer_PO_Number_Key = "Customer PO#"
    Customer_PO_Number1_Key = "Customer Po:"
    Delivery_number_Key = "Delivery #:"
    Shipment_Number_Key = "Shipment #:"
    Shipment_Number1_Key = "ShipmentNumber #"
    Delivery_time_Key = "Delivery Time:"
    Delivery_time1_Key = "Delivery\nTime:"
    Delivery_time2_Key = "Delivery"
    pick_up_key = "City/State/Zip:\n"
    
    # Entities to return
    PO_NUMBER = "PO Number"
    DROPOFF_ADDRESS = "Dropoff Address"
    PICKUP_ADDRESS = "Pickup Address"
    DELIVERY_DATE = "Delivery Date"
    
    PO_FLAG = True
    PICKUP_FLAG = True
    DROPOFF_FLAG = True
    DELIVERY_DATE_FLAG = True
    
    po_list = []
    for key, value in entities_json.items():
        
        if key.startswith(Bill_of_Lading_Number_Key):
            try:
                bill_match = re.search(r'(\d{3,})', value)
                Bill_NO = bill_match.group(1)
                po_list.append(Bill_NO)
            except:
                pass
        if key.startswith(Master_Bill_of_Lading_Number_Key):
            try:
                Master_bill_match = re.search(r'(\d{3,})', value)
                Master_Bill = Master_bill_match.group(1)
                po_list.append(Master_Bill)
            except:
                pass
        if key.startswith(Customer_PO_Number_Key) or key.startswith(Customer_PO_Number1_Key):
            try:
                Customer_po_match = re.search(r'(\d{3,})', value)
                Customer_po = Customer_po_match.group(1)
                po_list.append(Customer_po)
            except:
                pass
        if key.startswith(Delivery_number_Key):
            try:
                Delivery_num_match = re.search(r'(\d{3,})', value)
                Delivery_num = Delivery_num_match.group(1)
                po_list.append(Delivery_num)
            except:
                pass
        if key.startswith(Shipment_Number_Key) or key.startswith(Shipment_Number1_Key):
            try:
                Shipment_Number_match = re.search(r'(\d{3,})', value)
                Shipment_Number = Shipment_Number_match.group(1)
                po_list.append(Shipment_Number)
                #print(po_list)
            except:
                pass
            
            #PO_num = Bill_NO + "@" + Master_Bill + "@" + Customer_po + "@" + Delivery_num + "@" + Shipment_Number
            try:
                if PO_FLAG:
                    parsed_data[PO_NUMBER] = '@'.join(po_list)
                    PO_FLAG = False
            except:
                pass
                
        if key.startswith(pick_up_key):
            pickup_match = re.search(r'(\w+\s[^\n]\w+\,\s[A-Z]{2}\s\d{5}|[^\s][A-Z]{0}\w+\,\s[A-Z]{2}\s\d{5})', value)
            try:
                if PICKUP_FLAG:
                    pick_up = pickup_match.group(1)
                    parsed_data[PICKUP_ADDRESS] = pick_up
                    PICKUP_FLAG = False
            except:
                pass
                    
        if key.startswith(Delivery_time_Key) or key.startswith(Delivery_time1_Key) or key.startswith(Delivery_time2_Key):
            #parsed_data[DELIVERY_DATE] = value
            try:
                date_match = re.search(r'(\d+/\d+/\d+)', value)
                if DELIVERY_DATE_FLAG:
                    date = date_match.group(1)
                    #print(date)
                    parsed_data[DELIVERY_DATE] = date.strip()
                    #print("sjdadf", parsed_data[DELIVERY_DATE])
                    DELIVERY_DATE_FLAG = False
            except:
                pass
            
    def get_po_number_from_raw_text(text):
        start_search_key = 'SHIP FROM'
        end_search_key = 'CARRIER'
        start_idx = text.find(start_search_key)
        end_idx = text.find(end_search_key)
        search_str = text[start_idx + len(start_search_key):end_idx].strip()
        try:
            pattern = re.compile(r'\d{3,}')
            match = pattern.findall(search_str.replace('\n', ' '))
            po = '@'.join(match)
            return po
        except:
            pass
    try:
        if PO_FLAG:
            po_num = get_po_number_from_raw_text(raw_text)
            parsed_data[PO_NUMBER] = po_num.strip()
            PO_FLAG = False
    except:
        pass
    
    def get_date_from_raw_text(raw_text):
        start_search_key = 'FACILITY CHECKOUT'
        start_idx = raw_text.find(start_search_key)
        search_str = raw_text[start_idx + len(start_search_key):start_idx + len(start_search_key) + 900].strip()
        search_str = search_str.replace('\n', ' ')
        #print(search_str)
        try:
            date_pattern_match = re.search(r"Delivery\s\w{4}:\s(\d+\/\d+\/\d+)", search_str)
            date = date_pattern_match.group(1)
            #print("DATE>>>>>>", date)
        except:
            date = "NA"
        return date
        #try:
            
        #except:
            #return "NA"
        #return date
    
    #print(DELIVERY_DATE_FLAG)
    if DELIVERY_DATE_FLAG:
        delivery_date = get_date_from_raw_text(raw_text)
        parsed_data[DELIVERY_DATE] = delivery_date.strip()
        DELIVERY_DATE_FLAG = False
        
    def get_date_from_raw_text2(raw_text):
        start_search_key = 'FACILITY CHECKOUT'
        start_idx = raw_text.find(start_search_key)
        search_str = raw_text[start_idx + len(start_search_key):start_idx + len(start_search_key) + 900].strip()
        search_str = search_str.replace('\n', ' ')
        #print(search_str)
        try:
            date_pattern_match = re.search(r"Delivery\s(\d+\/\d+\/\d+)", search_str)
            date = date_pattern_match.group(1)
            #print("DATE$$$$", date)
        except:
            date = "NA"
        return date
        
    if parsed_data[DELIVERY_DATE] == "NA":
        try:
            delivery_dt = get_date_from_raw_text2(raw_text)
            parsed_data[DELIVERY_DATE] = delivery_dt.strip()
        except:
            pass
    
    def get_date_from_raw_text3(raw_text):
        start_search_key = 'FACILITY CHECKOUT'
        start_idx = raw_text.find(start_search_key)
        search_str = raw_text[start_idx + len(start_search_key):start_idx + len(start_search_key) + 900].strip()
        search_str = search_str.replace('\n', ' ')
        #print(search_str)
        try:
            date_pattern_match = re.search(r"Delivery\s\w{4}\s(\d+\/\d+\/\d+)", search_str)
            date = date_pattern_match.group(1)
            #print("DATE>>>>>>", date)
        except:
            date = "NA"
        return date
    
    if parsed_data[DELIVERY_DATE] == "NA":
        try:
            delivery_dt1 = get_date_from_raw_text3(raw_text)
            parsed_data[DELIVERY_DATE] = delivery_dt1.strip()
        except:
            pass
        
    def get_date_from_raw_text4(raw_text):
        start_search_key = 'FACILITY CHECKOUT'
        start_idx = raw_text.find(start_search_key)
        search_str = raw_text[start_idx + len(start_search_key):].strip()
        search_str = search_str.replace('\n', ' ')
        #print(search_str)
        try:
            date_pattern = re.compile(r"\d+\/\d+\/\d+")
            date_list = date_pattern.findall(search_str)
            print("DATEList%%%%%%%", date_list)
        except:
            date = "NA"
        return date_list
    
    if parsed_data[DELIVERY_DATE] == "NA":
        try:
            delivery_dt2 = get_date_from_raw_text4(raw_text)
            parsed_data[DELIVERY_DATE] = delivery_dt2[-1].strip()
        except:
            pass         
        
    def get_city_state_zip(text):
        start_search_key = 'SHIP FROM'
        end_search_key = 'Customer Order Information'
        start_idx = text.find(start_search_key)
        end_idx = text.find(end_search_key)
        search_str = text[start_idx + len(start_search_key):end_idx].strip()
        #search_str = search_str.replace('\n', ' ')
        #print(search_str)
        
        pattern = re.compile(r'\w+\s[^\n]\w+\,\s[A-Z]{2}\s\d{5}|[^\s][A-Z]{0}\w+\,\s[A-Z]{2}\s\d{5}')
        match = pattern.findall(search_str)
        #print(match)
        if match:
            return match
        else:
            return ""
        
    def get_address(raw_text):
        
        
    zip_list = get_city_state_zip(raw_text)
    #print(zip_list)
    try:
        if PICKUP_FLAG:
            parsed_data[PICKUP_ADDRESS] = zip_list[0].strip()
        if DROPOFF_FLAG:
            parsed_data[DROPOFF_ADDRESS] = zip_list[-1].strip()
    except:
        pass
            
        
        

            
    ner_string = parse_ner_json_to_string(parsed_data)
    return [200, ner_string]
    #return parsed_data


In [ ]:
data = data_files[[*data_files][0]]

In [ ]:
a = niagara_format1_doc_ai_ner_parser(data['raw_text'].decode(), data['entities'])

In [ ]:
extract_data = []
for file,data in data_files.items():
    a = niagara_format1_doc_ai_ner_parser(data['raw_text'].decode(), data['entities'])
    extract_data.append(a)

In [ ]:
df = pd.DataFrame(extract_data)

In [ ]:
df

In [ ]:
data_files[[*data_files][37]]['raw_text']

In [ ]:
data_files[[*data_files][208]]['entities']

In [ ]:
data_files[[*data_files][12]]['raw_text']

In [ ]:
def niagara_format1_doc_ai_ner_parser1(raw_text, entities_json):
    parsed_data = {}
    
    #Look up keys
    Bill_of_Lading_Number_Key = "Bill of Lading Number:"
    Master_Bill_of_Lading_Number_Key = "Master Bill of Lading Number:"
    Customer_PO_Number_Key = "Customer PO#"
    Customer_PO_Number1_Key = "Customer Po:"
    Delivery_number_Key = "Delivery #:"
    Shipment_Number_Key = "Shipment #:"
    Shipment_Number1_Key = "ShipmentNumber #"
    Delivery_time_Key = "Delivery Time:"
    Delivery_time1_Key = "Delivery\nTime:"
    Delivery_time2_Key = "Delivery"
    pick_up_key = "City/State/Zip:\n"
    
    # Entities to return
    PO_NUMBER = "PO Number"
    DROPOFF_ADDRESS = "Dropoff Address"
    PICKUP_ADDRESS = "Pickup Address"
    DELIVERY_DATE = "Delivery Date"
    
    PO_FLAG = True
    PICKUP_FLAG = True
    DROPOFF_FLAG = True
    DELIVERY_DATE_FLAG = True
    
    po_list = []
    for key, value in entities_json.items():
        
        if key.startswith(Bill_of_Lading_Number_Key):
            try:
                bill_match = re.search(r'(\d{3,})', value)
                Bill_NO = bill_match.group(1)
                po_list.append(Bill_NO)
            except:
                pass
        if key.startswith(Master_Bill_of_Lading_Number_Key):
            try:
                Master_bill_match = re.search(r'(\d{3,})', value)
                Master_Bill = Master_bill_match.group(1)
                po_list.append(Master_Bill)
            except:
                pass
        if key.startswith(Customer_PO_Number_Key) or key.startswith(Customer_PO_Number1_Key):
            try:
                Customer_po_match = re.search(r'(\d{3,})', value)
                Customer_po = Customer_po_match.group(1)
                po_list.append(Customer_po)
            except:
                pass
        if key.startswith(Delivery_number_Key):
            try:
                Delivery_num_match = re.search(r'(\d{3,})', value)
                Delivery_num = Delivery_num_match.group(1)
                po_list.append(Delivery_num)
            except:
                pass
        if key.startswith(Shipment_Number_Key) or key.startswith(Shipment_Number1_Key):
            try:
                Shipment_Number_match = re.search(r'(\d{3,})', value)
                Shipment_Number = Shipment_Number_match.group(1)
                po_list.append(Shipment_Number)
            except:
                pass
            try:
                if PO_FLAG:
                    parsed_data[PO_NUMBER] = '@'.join(po_list)
                    PO_FLAG = False
            except:
                pass
                
        if key.startswith(pick_up_key):
            pickup_match = re.search(r'(\w+\s[^\n]\w+\,\s[A-Z]{2}\s\d{5}|[^\s][A-Z]{0}\w+\,\s[A-Z]{2}\s\d{5})', value)
            try:
                if PICKUP_FLAG:
                    pick_up = pickup_match.group(1)
                    parsed_data[PICKUP_ADDRESS] = pick_up
                    PICKUP_FLAG = False
            except:
                pass
                    
        if key.startswith(Delivery_time_Key) or key.startswith(Delivery_time1_Key) or key.startswith(Delivery_time2_Key):
            try:
                date_match = re.search(r'(\d+/\d+/\d+)', value)
                if DELIVERY_DATE_FLAG:
                    date = date_match.group(1)
                    parsed_data[DELIVERY_DATE] = date.strip()
                    DELIVERY_DATE_FLAG = False
            except:
                pass
            
    def get_po_number_from_raw_text(text):
        start_search_key = 'SHIP FROM'
        end_search_key = 'CARRIER'
        start_idx = text.find(start_search_key)
        end_idx = text.find(end_search_key)
        search_str = text[start_idx + len(start_search_key):end_idx].strip()
        try:
            pattern = re.compile(r'\d{3,}')
            match = pattern.findall(search_str.replace('\n', ' '))
            po = '@'.join(match)
            return po
        except:
            pass
    try:
        if PO_FLAG:
            po_num = get_po_number_from_raw_text(raw_text)
            parsed_data[PO_NUMBER] = po_num.strip()
            PO_FLAG = False
    except:
        pass
    
    def get_date_from_raw_text(raw_text):
        start_search_key = 'FACILITY CHECKOUT'
        start_idx = raw_text.find(start_search_key)
        search_str = raw_text[start_idx + len(start_search_key):start_idx + len(start_search_key) + 900].strip()
        search_str = search_str.replace('\n', ' ')
        try:
            date_pattern_match = re.search(r"Delivery\s\w{4}:\s(\d+\/\d+\/\d+)", search_str)
            date = date_pattern_match.group(1)
        except:
            date = "NA"
        return date

    if DELIVERY_DATE_FLAG:
        delivery_date = get_date_from_raw_text(raw_text)
        parsed_data[DELIVERY_DATE] = delivery_date.strip()
        DELIVERY_DATE_FLAG = False
        
    def get_date_from_raw_text2(raw_text):
        start_search_key = 'FACILITY CHECKOUT'
        start_idx = raw_text.find(start_search_key)
        search_str = raw_text[start_idx + len(start_search_key):start_idx + len(start_search_key) + 900].strip()
        search_str = search_str.replace('\n', ' ')
        try:
            date_pattern_match = re.search(r"Delivery\s(\d+\/\d+\/\d+)", search_str)
            date = date_pattern_match.group(1)
        except:
            date = "NA"
        return date
        
    if parsed_data[DELIVERY_DATE] == "NA":
        try:
            delivery_dt = get_date_from_raw_text2(raw_text)
            parsed_data[DELIVERY_DATE] = delivery_dt.strip()
        except:
            pass
    
    def get_date_from_raw_text3(raw_text):
        start_search_key = 'FACILITY CHECKOUT'
        start_idx = raw_text.find(start_search_key)
        search_str = raw_text[start_idx + len(start_search_key):start_idx + len(start_search_key) + 900].strip()
        search_str = search_str.replace('\n', ' ')
        try:
            date_pattern_match = re.search(r"Delivery\s\w{4}\s(\d+\/\d+\/\d+)", search_str)
            date = date_pattern_match.group(1)
        except:
            date = "NA"
        return date
    
    if parsed_data[DELIVERY_DATE] == "NA":
        try:
            delivery_dt1 = get_date_from_raw_text3(raw_text)
            parsed_data[DELIVERY_DATE] = delivery_dt1.strip()
        except:
            pass
        
    def get_date_from_raw_text4(raw_text):
        start_search_key = 'FACILITY CHECKOUT'
        start_idx = raw_text.find(start_search_key)
        search_str = raw_text[start_idx + len(start_search_key):].strip()
        search_str = search_str.replace('\n', ' ')
        try:
            date_pattern = re.compile(r"\d+\/\d+\/\d+")
            date_list = date_pattern.findall(search_str)
        except:
            date = "NA"
        return date_list
    
    if parsed_data[DELIVERY_DATE] == "NA":
        try:
            delivery_dt2 = get_date_from_raw_text4(raw_text)
            parsed_data[DELIVERY_DATE] = delivery_dt2[-1].strip()
        except:
            pass           
        
    def get_city_state_zip(text):
        start_search_key = 'SHIP FROM'
        end_search_key = 'Customer Order Information'
        start_idx = text.find(start_search_key)
        end_idx = text.find(end_search_key)
        search_str = text[start_idx + len(start_search_key):end_idx].strip()
        
        pattern = re.compile(r'\w+\s[^\n]\w+\,\s[A-Z]{2}\s\d{5}|[^\s][A-Z]{0}\w+\,\s[A-Z]{2}\s\d{5}')
        match = pattern.findall(search_str)
        if match:
            return match
        else:
            return ""
        
    zip_list = get_city_state_zip(raw_text)
    try:
        if PICKUP_FLAG:
            parsed_data[PICKUP_ADDRESS] = zip_list[0].strip()
        if DROPOFF_FLAG:
            parsed_data[DROPOFF_ADDRESS] = zip_list[-1].strip()
    except:
        pass
            
            
    ner_string = parse_ner_json_to_string(parsed_data)
    return [200, ner_string]


In [ ]:
extract_data = []
for file,data in data_files.items():
    a = niagara_format1_doc_ai_ner_parser1(data['raw_text'].decode(), data['entities'])
    extract_data.append(a)

In [ ]:
df = pd.DataFrame(extract_data)

In [ ]:
df